## Librerias

In [2]:
from werkzeug.wrappers import Request, Response
#from flask import Flask, render_template
from jinja2 import Template

#from datetime import date
from datetime import datetime, timedelta

import collections
from amigocloud import AmigoCloud

from datetime import datetime

## Conectar a AmigoCloud

In [3]:
# CONECTAR A AMIGOCLOUD
acceso = open('../api_key/api.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

## Datos de crédito

In [4]:
semana = 'S07'

In [5]:
query = {'query': 'select insp.canhero, prop.propiedad, lote.lote, gal.s3_filename, gal.amigocloud_url foto\
         from dataset_307560 insp\
         inner join dataset_307562 prop on insp.amigo_id=prop.insp_ref_prop\
         inner join dataset_307563 lote on prop.amigo_id=lote.prop_ref_lote\
         inner join gallery_46845 gal on lote.amigo_id=gal.source_amigo_id\
         where semana_1ra = \'{sem}\' order by canhero'.format(sem=semana)}
select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/32672/sql', query)
data = select['data']
#credito = collections.namedtuple("credito", credito_data.keys())(*credito_data.values())
print('end')

end


In [6]:
data

[{'canhero': '11227 / QUIROZ OLGUIN DENICE',
  'propiedad': '741 / PROPIEDAD TARIJA II--OLGUIN',
  'lote': 'L2.1',
  's3_filename': 'WhatsApp Image 2023-02-17 at 15.25.54.jpeg',
  'foto': 'app.amigocloud.com/api/v1/related_tables/46845/files/4906259a5149492fbe433cefe7b2448a/360cd77e059e4ab099170d66b9b3e2e8/WhatsApp%20Image%202023-02-17%20at%2015.25.54.jpeg'},
 {'canhero': '1218 / BALCAZAR ALCANTARA JOSE LOLY',
  'propiedad': '510 / LA BELLA & SANTA ROSA I & VILLA FLORES',
  'lote': 'L5.2',
  's3_filename': 'LOLY.jpeg',
  'foto': 'app.amigocloud.com/api/v1/related_tables/46845/files/a9d29cc46f17430f893240726c966d64/68edd3077c35424aa9259aec7dc64da2/LOLY.jpeg'},
 {'canhero': '18028 / RIVERO GUTIERREZ FRANCISCO',
  'propiedad': '705 / LOMAS DE BIBOSI RIVERO',
  'lote': 'L1.2',
  's3_filename': 'WhatsApp Image 2023-02-17 at 15.06.54 (1).jpeg',
  'foto': 'app.amigocloud.com/api/v1/related_tables/46845/files/35206a63a45f407cb9ed860e9efdd7d8/38bfbb28abec4852a3fce8ff9fcb33f5/WhatsApp%20Image%20

In [ ]:
from docxtpl import DocxTemplate
import docxtpl
from docx.shared import Mm

import requests

doc = DocxTemplate("templates/template_1ra_fotos.docx")

#descargar fotos y generar lista InlineImage
lista_fotos_inline = []
for foto in data:
    response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['s3_filename'])
    file = open('fotos/' + foto['s3_filename'], "wb")
    file.write(response.content)
    file.close()
    lista_fotos_inline.append({'foto': docxtpl.InlineImage(doc, image_descriptor='fotos/' + foto['s3_filename'], width=Mm(120)), 'canhero': foto['canhero']})
    print(foto['s3_filename'])

context = {'fotos':lista_fotos_inline}

doc.render(context)
file_name = 'SEMANA - ' + semana
doc.save('_' + file_name + '.docx')

WhatsApp Image 2023-02-17 at 15.25.54.jpeg
LOLY.jpeg
WhatsApp Image 2023-02-17 at 15.06.54 (1).jpeg
WhatsApp Image 2023-02-17 at 15.06.56 (1).jpeg
CANDIA.jpeg
CANDIA 2.jpeg
WhatsApp Image 2023-02-17 at 15.06.55.jpeg
WhatsApp Image 2023-02-17 at 15.49.04.jpeg
WhatsApp Image 2023-02-17 at 15.49.05.jpeg
WhatsApp Image 2023-02-17 at 15.47.21 (2).jpeg
Imagen de WhatsApp 2023-02-18 a las s09.49.34.jpg
WhatsApp Image 2023-02-17 at 1.31.04 PM.jpeg
WhatsApp Image 2023-02-18 at 11.46.27.jpeg
WhatsApp Image 2023-02-17 at 15.48.19.jpeg
WhatsApp Image 2023-02-17 at 15.48.21 (1).jpeg
WhatsApp Image 2023-02-17 at 15.47.20.jpeg
WhatsApp Image 2023-02-10 at 09.59.06 (1)_NpgjNUa.jpeg
WhatsApp Image 2023-02-17 at 15.23.12.jpeg
WhatsApp Image 2023-02-17 at 1.31.03 PM.jpeg
WhatsApp Image 2023-02-17 at 1.31.02 PM (2).jpeg
HUANCA.jpeg
WhatsApp Image 2023-02-16 at 9.43.39 AM.jpeg
WhatsApp Image 2023-02-16 at 9.43.39 AM (1).jpeg
AIDE LINARES.jpeg
WhatsApp Image 2023-02-03 at 16.35.29 (2).jpeg
